In [ ]:
from power_flow.core.bus import Bus
from power_flow.core.line import Line
import pyomo.environ as pyo


In [ ]:
# Define bus and line data
buses = ["Dallas", "Houston", "Austin", "San Antonio"]
slack_bus = "Dallas"

# Net injections in MW (generation minus load)
P_dict = {
    "Dallas": 400.0,
    "Houston": -250.0,
    "Austin": -100.0,
    "San Antonio": -50.0,
}

# Lat lon coordinates for each city
bus_coords = {
    "Dallas": (32.78, -96.80),
    "Houston": (29.76, -95.37),
    "Austin": (30.27, -97.74),
    "San Antonio": (29.42, -98.49),
}

# Create Bus objects
def make_bus(name):
    lat, lon = bus_coords[name]
    angle_guess = 0.0 if name == slack_bus else 0.0
    return Bus(name=name, x_coord=lon, y_coord=lat, p_injection=P_dict[name], angle=angle_guess)

bus_objects = {name: make_bus(name) for name in buses}

# Line data: (from, to, reactance)
lines_data = [
    ("Dallas", "Austin", 0.12),
    ("Dallas", "Houston", 0.20),
    ("Austin", "Houston", 0.10),
    ("Austin", "San Antonio", 0.08),
    ("Houston", "San Antonio", 0.25),
]

line_objects = []
for idx, (bus_from, bus_to, reactance) in enumerate(lines_data, start=1):
    line_objects.append(
        Line(
            name=f"Line {idx}: {bus_from}->{bus_to}",
            bus_from=bus_objects[bus_from],
            bus_to=bus_objects[bus_to],
            x=reactance,
            rating_mw=1000.0,
        )
    )

line_pairs = [(line.bus_from.name, line.bus_to.name) for line in line_objects]
reactance_map = {(line.bus_from.name, line.bus_to.name): line.x for line in line_objects}


In [ ]:
# Build Pyomo model for DC power flow
model = pyo.ConcreteModel()
model.BUS = pyo.Set(initialize=buses)
model.LINES = pyo.Set(dimen=2, initialize=line_pairs)

# Parameters
model.P = pyo.Param(model.BUS, initialize=P_dict)
model.x = pyo.Param(model.LINES, initialize=reactance_map)

# Variables (bus voltage angles in radians)
model.theta = pyo.Var(model.BUS, bounds=(-3.14, 3.14))
model.theta[slack_bus].fix(0.0)

# Power balance constraints
def power_balance_rule(m, b):
    outgoing = sum((m.theta[b] - m.theta[t]) / m.x[(b, t)] for (f, t) in m.LINES if f == b)
    incoming = sum((m.theta[b] - m.theta[f]) / m.x[(f, b)] for (f, t) in m.LINES if t == b)
    return outgoing + incoming == m.P[b]

model.power_balance = pyo.Constraint(model.BUS, rule=power_balance_rule)

# Dummy objective (feasibility problem)
model.obj = pyo.Objective(expr=0)

# Solve
glpk = pyo.SolverFactory("glpk")
results = glpk.solve(model, tee=False)

angles = {b: pyo.value(model.theta[b]) for b in model.BUS}
for name, bus in bus_objects.items():
    bus.angle = angles[name]
angles


In [ ]:
# Calculate line flows based on solved bus angles
line_flows = {}
for (f, t) in model.LINES:
    flow = (model.theta[f] - model.theta[t]) / model.x[(f, t)]
    line_flows[f"{f}->{t}"] = pyo.value(flow)
line_flows
